In [1]:
import torch
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset
from wechsel import WECHSEL, load_embeddings

/cw/dtaijupiter/NoCsBack/dtai/pieterd/projects/transtokenizer/.env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/cw/dtaijupiter/NoCsBack/dtai/pieterd/projects/transtokenizer/.env/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
import os
os.environ["XDG_CACHE_HOME"] = "HF_HOME=/cw/dtaijupiter/NoCsBack/dtai/pieterd/.cache"

In [3]:

source_tokenizer = AutoTokenizer.from_pretrained("mistralai/mistral-7b-v0.1")
model = AutoModel.from_pretrained("mistralai/mistral-7b-v0.1")

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.11s/it]


In [4]:
target_tokenizer = AutoTokenizer.from_pretrained("yhavinga/gpt-neo-1.3B-dutch")

In [6]:

wechsel = WECHSEL(
    load_embeddings("en"),
    load_embeddings("nl"),
    bilingual_dictionary="dutch",
    
)

In [7]:

target_embeddings, info = wechsel.apply(
    source_tokenizer,
    target_tokenizer,
    model.get_input_embeddings().weight.detach().numpy(),
)

100%|██████████| 50/50 [00:33<00:00,  1.48it/s]


In [8]:

model.get_input_embeddings().weight.data = torch.from_numpy(target_embeddings)
model.config.vocab_size = len(target_embeddings)

In [17]:
model.save_pretrained("../wechsel-dutch/")

In [18]:
target_tokenizer.save_pretrained("../wechsel-dutch/") 

('../wechsel-dutch/tokenizer_config.json',
 '../wechsel-dutch/special_tokens_map.json',
 '../wechsel-dutch/vocab.json',
 '../wechsel-dutch/merges.txt',
 '../wechsel-dutch/added_tokens.json',
 '../wechsel-dutch/tokenizer.json')